Danil Timofeev, AI01 \
d.timofeev@innopolis.university

# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [9]:
def load_fivegrams(file_path):
    fivegrams = {}
    with open(file_path, "r") as file:
        for line in file:
            parts = line.strip().split()
            frequency = int(parts[0])
            fivegram = tuple(parts[1:])
            fivegrams[fivegram] = frequency
    return fivegrams


def generate_candidate_words(word, dictionary, max_distance=1):
    """Generate candidate corrections for a word within a given edit distance."""
    candidates = set()
    alphabet = "abcdefghijklmnopqrstuvwxyz"

    # Deletion
    candidates.update(word[:i] + word[i + 1 :] for i in range(len(word)))
    # Transposition
    candidates.update(
        word[:i] + word[i + 1] + word[i] + word[i + 2 :] for i in range(len(word) - 1)
    )
    # Alteration
    candidates.update(
        word[:i] + c + word[i + 1 :] for i in range(len(word)) for c in alphabet
    )
    # Insertion
    candidates.update(
        word[:i] + c + word[i:] for i in range(len(word) + 1) for c in alphabet
    )

    return [w for w in candidates if w in dictionary]


def find_best_correction(word, context, fivegrams, dictionary):
    candidates = generate_candidate_words(word, dictionary)
    # Include the original word in case it's the best choice
    candidates.append(word)
    max_frequency = -1
    best_candidate = word

    for candidate in candidates:
        for i in range(max(0, len(context) - 4), len(context)):
            potential_context = (
                context[max(0, i - 4) : i]
                + [candidate]
                + context[i + 1 : min(i + 5, len(context))]
            )
            potential_fivegram = tuple(potential_context)
            frequency = fivegrams.get(potential_fivegram, 0)
            if frequency > max_frequency:
                max_frequency = frequency
                best_candidate = candidate

    return best_candidate


def correct_spelling(input_strings, fivegrams):
    # Build a dictionary from the fivegrams
    dictionary = set(word for fivegram in fivegrams for word in fivegram)

    corrected_strings = []
    for input_string in input_strings:
        words = input_string.split()
        corrected_words = []
        for i, word in enumerate(words):
            if word not in dictionary:
                context = words[max(0, i - 4) : i + 5]  # Get 4 words before and after
                correction = find_best_correction(word, context, fivegrams, dictionary)
                corrected_words.append(correction)
            else:
                corrected_words.append(word)
        corrected_strings.append(" ".join(corrected_words))
    return corrected_strings


['his is your example string with some misspelled words']


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

1. Using a Five-gram Model for Context \
    Decision: The code uses a five-gram model to provide context for spelling corrections.

    Justification: Five-grams strike a balance between offering a rich context and managing data sparsity. They provide enough surrounding words to make informed decisions about the most probable correction based on not just the immediate adjacent words but a broader sentence context. This is particularly useful for distinguishing between words that are commonly confused or for choosing the correct spelling in context-sensitive situations.

2. Simple Edit Operations for Generating Candidates \
    Decision: The candidate generation function (generate_candidate_words) creates possible corrections through four types of edit operations: deletions, transpositions, alterations, and insertions.

    Justification: These operations are based on the most common types of spelling errors, reflecting real-world typing mistakes. Limiting the generation to these operations keeps the candidate set relevant and manageable, improving efficiency while covering a wide range of potential errors.

3. Inclusion of the Original Word in Candidates \
    Decision: The original word is included in the set of candidates for correction.

    Justification: Including the original word accounts for cases where the word may not be present in the dictionary (used for generating candidates) but is still correct. This is important for handling proper nouns, technical terms, or newly coined words not captured in the dataset.

4. Selection of Best Correction Based on Frequency \
    Decision: The correction chosen is the one that, when inserted into the context, forms the most frequent five-gram.

    Justification: This approach leverages the frequency information from the five-gram dataset to choose corrections that are not only contextually appropriate but also statistically probable. It's based on the assumption that more frequently occurring word sequences are more likely to be correct, a principle grounded in the statistical nature of language use.

## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [11]:
import pandas as pd

# Example usage
file_path = "../data/fivegrams.txt"
input_strings = pd.read_csv('../data/test/test.csv')['Misspelled'].str.lower().tolist()
fivegrams = load_fivegrams(file_path)
corrected_strings = correct_spelling(input_strings, fivegrams)

print(corrected_strings)

["i can't believe how beautiful this place is", 'their going to join us later at the resturant.', 'she has a very unique way of thinking', "it's definitely holder than yesterday", 'could you be more pacific about what happened', 'he excersises regularly to stay healthy', 'the accommodation was not what we expected', 'i relieved a letter in the mail', 'the museum is closed on wendesday.', 'their new policy could affect us ally', "it's an opportunity i can't miss", 'we need to dress the issue immediately', 'i recommend taking a different routes', 'hers always had trouble with rythm.', 'this is a separate issue from what we discussed', 'lets keep it confidential for now just between you and ib', 'she was greatful for the help she received', 'i appreciate your patience during this time', 'he preferred the blue shirt over the red ones', 'please ensure all your belonging are securred.', 'im looking forward to receiving your feedbacks', 'the argument was irrelevant to the discussions', 'were 